In [1]:
import pandas as pd
import numpy as np
import re
import string

from expertai.nlapi.cloud.client import ExpertAiClient
import os

In [2]:
filename = "Womens Clothing E-Commerce Reviews.csv"

In [3]:
reviews = pd.read_csv("../input/" + filename)

In [4]:
reviews["Class Name"].unique(),len(reviews["Class Name"].unique())

(array(['Intimates', 'Dresses', 'Pants', 'Blouses', 'Knits', 'Outerwear',
        'Lounge', 'Sweaters', 'Skirts', 'Fine gauge', 'Sleep', 'Jackets',
        'Swim', 'Trend', 'Jeans', 'Legwear', 'Shorts', 'Layering',
        'Casual bottoms', nan, 'Chemises'], dtype=object),
 21)

In [5]:
os.environ["EAI_USERNAME"] = 'ambarish.ganguly@gmail.com'
os.environ["EAI_PASSWORD"] = 'Ambarish@1234'

In [6]:
def clean_text(text):
    '''Make text lowercase,remove punctuation
    .'''
    text = str(text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\r', ' ', text)
    return text

In [7]:
client = ExpertAiClient()
language ="en"

In [8]:
def sentiment_overall(client, text, language):
    try:
        input_text = str(text)
        document = client.specific_resource_analysis(
        body={"document": {"text": input_text}}, 
        params={'language':  language, 'resource': 'sentiment'
       })
        return document.sentiment.overall
    except Exception as e: 
        print(str(e) +": " + str(text))

In [9]:
def emotional_traits(client, text, language,taxonomy = "emotional-traits"):
    input_text = str(text)
    emotional_categories = ""
    output = client.classification(body={"document": {"text": input_text}}, 
                                   params={'taxonomy': taxonomy, 'language': language})

    for category in output.categories:
        if(emotional_categories == ""):
            emotional_categories = emotional_categories + str(category.hierarchy[-1])
        else:
            emotional_categories = emotional_categories + " " + str(category.hierarchy[-1])
    
    return(emotional_categories)

In [10]:
def relevants(client, text, language):
    input_text = str(text)
    concepts = []
    document  = client.specific_resource_analysis(body={"document": {"text": input_text}}, 
                                   params={'language': language, 'resource': 'relevants'})

    for category in document.main_syncons:        
        concepts.append(category.lemma)
    
    return(concepts)

In [11]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [12]:
def create_sentiment_traits(category,number_of_rows,filename):
    reviews_cat = reviews[reviews["Class Name"] == category]
    
    reviews_cat["Review Text"] = reviews_cat["Review Text"].apply(lambda x: clean_text(x))
    
    r = reviews_cat[:number_of_rows]
    
    r["sentiment_overall"] = r["Review Text"].apply(lambda x: sentiment_overall(client, x, language))
    
    r["emotional_traits"] = r["Review Text"].apply(lambda x: emotional_traits(client, x, language))
    
    
    taxonomy = "behavioral-traits"
    r["behavioral_traits"] = r["Review Text"].apply(lambda x: emotional_traits(client, x, language,taxonomy))
    
    r["concepts"] = r["Review Text"].apply(lambda x: relevants(client, x, language))
    
    r = r.drop(columns =['Unnamed: 0'])
    
    r.to_csv("../output/" + filename,index = False)

In [13]:
create_sentiment_traits("Dresses",500,"dress_sentiment_traits.csv")

<ipython-input-12-d02defc4befe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_cat["Review Text"] = reviews_cat["Review Text"].apply(lambda x: clean_text(x))
<ipython-input-12-d02defc4befe>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r["sentiment_overall"] = r["Review Text"].apply(lambda x: sentiment_overall(client, x, language))
<ipython-input-12-d02defc4befe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [ ]:
create_sentiment_traits("Intimates",500,"intimates_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Pants",500,"pants_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Jeans",500,"jeans_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Swim",500,"swim_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Blouses",500,"blouses_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Knits",500,"knits_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Knits",500,"knits_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Outerwear",500,"outerwear_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Lounge",500,"lounge_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Sweaters",500,"sweaters_sentiment_traits.csv")

In [ ]:
create_sentiment_traits("Sweaters",500,"sweaters_sentiment_traits.csv")